# Rhyzopertha dominica

## get data

data were shared by Oppert, Brenda <Brenda.Oppert@ARS.USDA.GOV>

download the genome and RNA-seq data from https://de.cyverse.org/de/ with Cyberduck. The software Cyberduck is not user-friendly. The downloaded files are easily get broken, many times larger than expected. Too much errors. Need to be careful with it.

### fix sequence problem
get a error while running Trinity, in the Trimmamatic step, base length and quality length do not match for file `Adult_filtered_6-25-2012-USDA_NoIndex_L005_R1.fastq` and `Adult_filtered_6-25-2012-USDA_NoIndex_L005_R2.fastq`. Fix that.

need to be very careful when downloading the file with the Cyberduck program.
files
```
Adult_filtered_6-25-2012-USDA_NoIndex_L005_R1.fastq
Adult_filtered_6-25-2012-USDA_NoIndex_L005_R2.fastq
Carcass-1_S3_L001_R1_001.fastq
Carcass-1_S3_L001_R2_001.fastq
Carcass-2_S5_L001_R1_001.fastq
Carcass-2_S5_L001_R2_001.fastq
Carcass-3_S9_L001_R1_001.fastq
Carcass-3_S9_L001_R2_001.fastq
Gut-1_S2_L001_R1_001.fastq
Gut-1_S2_L001_R2_001.fastq
Gut-2_S6_L001_R1_001.fastq
Gut-2_S6_L001_R2_001.fastq
Gut-3_S8_L001_R1_001.fastq
Gut-3_S8_L001_R2_001.fastq
Head-1_S1_L001_R1_001.fastq
Head-1_S1_L001_R2_001.fastq
Head-2_S4_L001_R1_001.fastq
Head-2_S4_L001_R2_001.fastq
Head-3_S7_L001_R1_001.fastq
Head-3_S7_L001_R2_001.fastq
ResExp_Rep2.fastq
ResExp_Rep3a.fastq
ResExp_Rep3b.fastq
ResExp_Rep3c.fastq
ResExp_Rep4.fastq
ResUxp_Rep2.fastq
ResUxp_Rep3.fastq
ResUxp_Rep4.fastq
SRR1130516_1.fastq
SRR1130516_2.fastq
SuscExp_Rep2.fastq
SuscExp_Rep3.fastq
SuscExp_Rep4.fastq
SuscUxp_Rep2.fastq
SuscUxp_Rep3.fastq
SuscUxp_Rep4.fastq

```

## de novo assembly

### Trinity

transfer the genome and RNA-seq data to Elf, folder `/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/`

```
srun -N 1 --partition long --qos long-award   -t 240:00:00 --pty bash #allocate one node for running Trinity
cd /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA
/gpfs/gpfs/scratch/xc278/p/trinityrnaseq-Trinity-v2.8.5/Trinity --seqType fq --max_memory 200G --single Adult_filtered_6-25-2012-USDA_NoIndex_L005_R1.fastq,Carcass-1_S3_L001_R1_001.fastq,Carcass-2_S5_L001_R1_001.fastq,Carcass-3_S9_L001_R1_001.fastq,Gut-1_S2_L001_R1_001.fastq,Gut-2_S6_L001_R1_001.fastq,Gut-3_S8_L001_R1_001.fastq,Head-1_S1_L001_R1_001.fastq,Head-2_S4_L001_R1_001.fastq,Head-3_S7_L001_R1_001.fastq,ResExp_Rep2.fastq,ResExp_Rep3a.fastq,ResExp_Rep3b.fastq,ResExp_Rep3c.fastq,ResExp_Rep4.fastq,ResUxp_Rep2.fastq,ResUxp_Rep3.fastq,ResUxp_Rep4.fastq,SuscExp_Rep2.fastq,SuscExp_Rep3.fastq,SuscExp_Rep4.fastq,SuscUxp_Rep2.fastq,SuscUxp_Rep3.fastq,SuscUxp_Rep4.fastq,Adult_filtered_6-25-2012-USDA_NoIndex_L005_R2.fastq,Carcass-1_S3_L001_R2_001.fastq,Carcass-2_S5_L001_R2_001.fastq,Carcass-3_S9_L001_R2_001.fastq,Gut-1_S2_L001_R2_001.fastq,Gut-2_S6_L001_R2_001.fastq,Gut-3_S8_L001_R2_001.fastq,Head-1_S1_L001_R2_001.fastq,Head-2_S4_L001_R2_001.fastq,Head-3_S7_L001_R2_001.fastq --CPU 36 --trimmomatic --no_normalize_reads --output /home/scratch/Trinity/ --min_kmer_cov 2 --run_as_paired


```

when running Trinity, the sequences names are not accepted by Trinity. Change with the scripts like
```
zcat input_reverse.fastq.gz | awk '{{print (NR%4 == 1) ? "@1_" ++i "/2": $0}}' | gzip -c > output_reverse.renamed.fastq.gz
```

### binpacker

copy the combined single.fa generated by Trinity to folder /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA

```bash
srun -N 1 --partition long --qos long-award   -t 240:00:00 --pty bash #allocate one node for running BinPacker
cd /home/scratch
cp /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/single.fa.gz ./
pigz -d single.fa.gz
/gpfs/gpfs/scratch/xc278/p/BinPacker/BinPacker_binary/BinPacker -s fa -p single -o ./BinPacker/ -u single.fa


```

conclusion: do not use BinPacker. It does not support multiple threads and would be slow. It is not updated for a long time.

### rnaSPAdes

```bash
srun -N 1 --partition long --qos long-award   -t 240:00:00 --pty bash #allocate one node for running rnaSPAdes
cd /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA
/home1/xc278/w/scratch/p/SPAdes/SPAdes-3.12.0-Linux/bin/rnaspades.py -o /home/scratch/SPAdes/ -t 36 -m 2000 --pe1-1 Adult_filtered_6-25-2012-USDA_NoIndex_L005_R1.fastq --pe1-2 Adult_filtered_6-25-2012-USDA_NoIndex_L005_R2.fastq --pe2-1 Carcass-1_S3_L001_R1_001.fastq --pe2-2 Carcass-1_S3_L001_R2_001.fastq --pe2-1 Carcass-2_S5_L001_R1_001.fastq --pe2-2 Carcass-2_S5_L001_R2_001.fastq --pe2-1 Carcass-3_S9_L001_R1_001.fastq --pe2-2 Carcass-3_S9_L001_R2_001.fastq --pe3-1 Gut-1_S2_L001_R1_001.fastq --pe3-2 Gut-1_S2_L001_R2_001.fastq --pe3-1 Gut-2_S6_L001_R1_001.fastq --pe3-2 Gut-2_S6_L001_R2_001.fastq --pe3-1 Gut-3_S8_L001_R1_001.fastq --pe3-2 Gut-3_S8_L001_R2_001.fastq --pe4-1 Head-1_S1_L001_R1_001.fastq --pe4-2 Head-1_S1_L001_R2_001.fastq --pe4-1 Head-2_S4_L001_R1_001.fastq --pe4-2 Head-2_S4_L001_R2_001.fastq --pe4-1 Head-3_S7_L001_R1_001.fastq --pe4-2 Head-3_S7_L001_R2_001.fastq --s1 ResExp_Rep2.fastq --s1 ResExp_Rep3a.fastq --s1 ResExp_Rep3b.fastq --s1 ResExp_Rep3c.fastq --s1 ResExp_Rep4.fastq --s1 ResUxp_Rep2.fastq --s1 ResUxp_Rep3.fastq --s1 ResUxp_Rep4.fastq --s1 SuscExp_Rep2.fastq --s1 SuscExp_Rep3.fastq --s1 SuscExp_Rep4.fastq --s1 SuscUxp_Rep2.fastq --s1 SuscUxp_Rep3.fastq --s1 SuscUxp_Rep4.fastq

```

do not use rnaSPAdes, too slow.

### Bridger
```
/gpfs/gpfs/scratch/xc278/p/bridger/Bridger_r2014-12-01/Bridger.pl --seqType fa --single /home/scratch/single.fa.gz -o /home/scratch/Bridger --CPU 36
```

### transAbyss

```
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/home1/xc278/p/anaconda3/lib/
/home1/xc278/p/anaconda3/envs/bio/bin/transabyss --se single.fa --length 200 --threads 36 --outdir transabyss --mpi 36 --pe /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/SRR1130516_1.fastq /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/SRR1130516_2.fastq
```

### conclusion for de novo

Trinity and TransAbyss are faster. Use them.

## HISTAT2 StringTie

```bash
cd /home/scratch
#build genome
/gpfs/gpfs/scratch/xc278/p/HISTAT2/hisat2-2.1.0/hisat2-build -p 36 /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/Genome/Rdo_Scaffolds.fa Rdo #build genome, 36 core
#align with HISTAT2
/gpfs/gpfs/scratch/xc278/p/HISTAT2/hisat2-2.1.0/hisat2 -p 36 -x Rdo -1 /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Adult_filtered_6-25-2012-USDA_NoIndex_L005_R1.fastq -2 /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Adult_filtered_6-25-2012-USDA_NoIndex_L005_R2.fastq -S adult.sam && samtools sort -@36 -o adult.bam adult.sam && rm adult.sam

/gpfs/gpfs/scratch/xc278/p/HISTAT2/hisat2-2.1.0/hisat2 -p 36 -x Rdo -1 /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Carcass-1_S3_L001_R1_001.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Carcass-2_S5_L001_R1_001.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Carcass-3_S9_L001_R1_001.fastq -2 /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Carcass-1_S3_L001_R2_001.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Carcass-2_S5_L001_R2_001.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Carcass-3_S9_L001_R2_001.fastq -S carcass.sam && samtools sort -@36 -o carcass.bam carcass.sam && rm carcass.sam

/gpfs/gpfs/scratch/xc278/p/HISTAT2/hisat2-2.1.0/hisat2 -p 36 -x Rdo -1 /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Gut-1_S2_L001_R1_001.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Gut-2_S6_L001_R1_001.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Gut-3_S8_L001_R1_001.fastq -2 /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Gut-1_S2_L001_R2_001.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Gut-2_S6_L001_R2_001.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Gut-3_S8_L001_R2_001.fastq -S gut.sam && samtools sort -@36 -o gut.bam gut.sam && rm gut.sam

/gpfs/gpfs/scratch/xc278/p/HISTAT2/hisat2-2.1.0/hisat2 -p 36 -x Rdo -1 /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Head-1_S1_L001_R1_001.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Head-2_S4_L001_R1_001.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Head-3_S7_L001_R1_001.fastq -2 /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Head-1_S1_L001_R2_001.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Head-2_S4_L001_R2_001.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/Head-3_S7_L001_R2_001.fastq
-S head.sam && samtools sort -@36 -o head.bam head.sam && rm head.sam

/gpfs/gpfs/scratch/xc278/p/HISTAT2/hisat2-2.1.0/hisat2 -p 36 -x Rdo -1 /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/SRR1130516_1.fastq -2 /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/SRR1130516_2.fastq -S srr.sam && samtools sort -@36 -o srr.bam srr.sam && rm srr.sam

/gpfs/gpfs/scratch/xc278/p/HISTAT2/hisat2-2.1.0/hisat2 -p 36 -x Rdo -U /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/ResExp_Rep2.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/ResExp_Rep3a.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/ResExp_Rep3b.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/ResExp_Rep3c.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/ResExp_Rep4.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/ResUxp_Rep2.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/ResUxp_Rep3.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/ResUxp_Rep4.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/SuscExp_Rep2.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/SuscExp_Rep3.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/SuscExp_Rep4.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/SuscUxp_Rep2.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/SuscUxp_Rep3.fastq,/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/SuscUxp_Rep4.fastq -S unpair.sam && samtools sort -@36 -o unpair.bam unpair.sam && rm unpair.sam

#stringtie
/gpfs/gpfs/scratch/xc278/p/StringTie/stringtie-1.3.6.Linux_x86_64/stringtie -p 36 adult.bam -o adult.gtf
/gpfs/gpfs/scratch/xc278/p/StringTie/stringtie-1.3.6.Linux_x86_64/stringtie -p 36 carcass.bam -o carcass.gtf
/gpfs/gpfs/scratch/xc278/p/StringTie/stringtie-1.3.6.Linux_x86_64/stringtie -p 36 gut.bam -o gut.gtf
/gpfs/gpfs/scratch/xc278/p/StringTie/stringtie-1.3.6.Linux_x86_64/stringtie -p 36 head.bam -o head.gtf
/gpfs/gpfs/scratch/xc278/p/StringTie/stringtie-1.3.6.Linux_x86_64/stringtie -p 36 srr.bam -o srr.gtf
/gpfs/gpfs/scratch/xc278/p/StringTie/stringtie-1.3.6.Linux_x86_64/stringtie -p 36 unpair.bam -o unpair.gtf

#merge gff
ls *.gtf >gtf_list
/gpfs/gpfs/scratch/xc278/p/StringTie/stringtie-1.3.6.Linux_x86_64/stringtie --merge -o /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/StringTie/20190625RdoStringTie.gff gtf_list

#get transcript sequence
/gpfs/gpfs/scratch/xc278/p/cufflinks/cufflinks-2.2.1.Linux_x86_64/gffread -g /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/Genome/Rdo_Scaffolds.fa -w  /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/StringTie/20190625RdoStringTie.fa /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/StringTie/20190625RdoStringTie.gff




```

## MAKER

similar to https://reslp.github.io/blog/My-MAKER-Pipeline/

Here, from BUSCO result, the quality of Genome is very high. Simplify the steps.

### split the genome to ~5kb and remove shorter than 200bp

Note: for this genome, some bases are "RYKMSWBDHV", replace with N

```python
file_genome = '/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/Genome/Rdo_Scaffolds.fa'

import os
from Bio import SeqIO
import re

n = 1
l = 0
outfolder = '/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/scfs/'
if not os.path.exists(outfolder):
    os.makedirs(outfolder)
outfile = open(os.path.join(outfolder,str(n)),'w')
for s in SeqIO.parse(file_genome,'fasta'):
    slen = len(s.seq)
    if slen <=200:
        continue
    l += slen
    seq = re.sub('[^ATCG]','N',str(s.seq))
    outfile.write('>'+s.id+'\n'+seq+'\n')
    if l >= 5000:
        outfile.close()
        n += 1
        l = 0
        outfile = open(os.path.join(outfolder,str(n)),'w')
outfile.close()
```

### combine EST sequences
remove sequences with Non-ATCG bases, generate `/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/ref/ref.rna.faa`

need to provide unique names, as there are some error reports from maker
```python
import os
from Bio import SeqIO

files_nt = '''/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/StringTie/20190625RdoStringTie.fa
/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/denovo/20190624Trinity.fasta
/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/denovo/20190625transabyss.fasta
'''.split()
file_nt = '/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/ref/ref.rna.faa'
fout = open(file_nt,'w')
n = 0
for f in files_nt:
    for s in SeqIO.parse(f,'fasta'):
        n += 1
        seq = str(s.seq)
        seq = seq.upper()
        if seq.count('A')+seq.count('T')+seq.count('C')+seq.count('G')+seq.count('*')+seq.count('N') == len(seq):
            fout.write('>EST'+str(n)+'\n'+str(s.seq)+'\n')
        else:
            print(s.id,'abnormal',s.seq)
fout.close()
```

### first round maker

```bash
source activate /gpfs/gpfs/scratch/xc278/maker2 #activate environment
cd /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step1
maker -CTL
```


modify the maker_opts.ctl file, changed lines are

Because maker cannot handle gff file of StringTie, do not use it.

```
genome=/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/Genome/Rdo_Scaffolds.fa #genome sequence (fasta file or fasta embeded in GFF3 file)
est=/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/ref/ref.rna.faa #set of ESTs or assembled mRNA-seq in fasta format
protein=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/uniprot_sprot.fasta  #protein sequence file in fasta format (i.e. from mutiple oransisms)
est2genome=1 #infer gene predictions directly from ESTs, 1 = yes, 0 = no
protein2genome=1 #infer predictions from protein homology, 1 = yes, 0 = no
```

```
#-----Genome (these are always required)
genome=/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/Genome/Rdo_Scaffolds.fa #genome sequence (fasta file or fasta embeded in GFF3 file)
organism_type=eukaryotic #eukaryotic or prokaryotic. Default is eukaryotic

#-----Re-annotation Using MAKER Derived GFF3
maker_gff= #MAKER derived GFF3 file
est_pass=0 #use ESTs in maker_gff: 1 = yes, 0 = no
altest_pass=0 #use alternate organism ESTs in maker_gff: 1 = yes, 0 = no
protein_pass=0 #use protein alignments in maker_gff: 1 = yes, 0 = no
rm_pass=0 #use repeats in maker_gff: 1 = yes, 0 = no
model_pass=0 #use gene models in maker_gff: 1 = yes, 0 = no
pred_pass=0 #use ab-initio predictions in maker_gff: 1 = yes, 0 = no
other_pass=0 #passthrough anyything else in maker_gff: 1 = yes, 0 = no

#-----EST Evidence (for best results provide a file for at least one)
est=/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/ref/ref.rna.faa #set of ESTs or assembled mRNA-seq in fasta format
altest= #EST/cDNA sequence file in fasta format from an alternate organism
est_gff= #aligned ESTs or mRNA-seq from an external GFF3 file
altest_gff= #aligned ESTs from a closly relate species in GFF3 format

#-----Protein Homology Evidence (for best results provide a file for at least one)
protein=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/uniprot_sprot.fasta  #protein sequence file in fasta format (i.e. from mutiple oransisms)
protein_gff=  #aligned protein homology evidence from an external GFF3 file

#-----Repeat Masking (leave values blank to skip repeat masking)
model_org=all #select a model organism for RepBase masking in RepeatMasker
rmlib= #provide an organism specific repeat library in fasta format for RepeatMasker
repeat_protein=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/te_proteins.fasta #provide a fasta file of transposable element proteins for RepeatRunner
rm_gff= #pre-identified repeat elements from an external GFF3 file
prok_rm=0 #forces MAKER to repeatmask prokaryotes (no reason to change this), 1 = yes, 0 = no
softmask=1 #use soft-masking rather than hard-masking in BLAST (i.e. seg and dust filtering)

#-----Gene Prediction
snaphmm= #SNAP HMM file
gmhmm= #GeneMark HMM file
augustus_species= #Augustus gene prediction species model
fgenesh_par_file= #FGENESH parameter file
pred_gff= #ab-initio predictions from an external GFF3 file
model_gff= #annotated gene models from an external GFF3 file (annotation pass-through)
est2genome=1 #infer gene predictions directly from ESTs, 1 = yes, 0 = no
protein2genome=1 #infer predictions from protein homology, 1 = yes, 0 = no
trna=0 #find tRNAs with tRNAscan, 1 = yes, 0 = no
snoscan_rrna= #rRNA file to have Snoscan find snoRNAs
unmask=0 #also run ab-initio prediction programs on unmasked sequence, 1 = yes, 0 = no

#-----Other Annotation Feature Types (features MAKER doesn't recognize)
other_gff= #extra features to pass-through to final MAKER generated GFF3 file

#-----External Application Behavior Options
alt_peptide=C #amino acid used to replace non-standard amino acids in BLAST databases
cpus=1 #max number of cpus to use in BLAST and RepeatMasker (not for MPI, leave 1 when using MPI)

#-----MAKER Behavior Options
max_dna_len=1000000000 #length for dividing up contigs into chunks (increases/decreases memory usage)
min_contig=1 #skip genome contigs below this length (under 10kb are often useless)

pred_flank=200 #flank for extending evidence clusters sent to gene predictors
pred_stats=0 #report AED and QI statistics for all predictions as well as models
AED_threshold=1 #Maximum Annotation Edit Distance allowed (bound by 0 and 1)
min_protein=0 #require at least this many amino acids in predicted proteins
alt_splice=0 #Take extra steps to try and find alternative splicing, 1 = yes, 0 = no
always_complete=0 #extra steps to force start and stop codons, 1 = yes, 0 = no
map_forward=0 #map names and attributes forward from old GFF3 genes, 1 = yes, 0 = no
keep_preds=0 #Concordance threshold to add unsupported gene prediction (bound by 0 and 1)

split_hit=10000 #length for the splitting of hits (expected max intron size for evidence alignments)
single_exon=1 #consider single exon EST evidence when generating annotations, 1 = yes, 0 = no
single_length=600 #min length required for single exon ESTs if 'single_exon is enabled'
correct_est_fusion=0 #limits use of ESTs in annotation to avoid fusion genes

tries=2 #number of times to try a contig if there is a failure for some reason
clean_try=0 #remove all data from previous run before retrying, 1 = yes, 0 = no
clean_up=0 #removes theVoid directory with individual analysis files, 1 = yes, 0 = no
TMP= #specify a directory other than the system default temporary directory for temporary files

```

generate scripts
```python
total_seg = 140
folder = '/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step1/'
commands = f'{folder}cmds.txt'
fout = open(commands,'w')
for n in range(1,total_seg):
    cmd = f'source activate /gpfs/gpfs/scratch/xc278/maker2 && cd /home/scratch && mkdir {n} && cd {n} && cp {folder}/maker*.ctl ./ && maker -genome /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/scfs/{n} -cpus 9 &&  gff3_merge -d *.maker.output/*master_datastore_index.log &&  mv *.gff {folder}/maker_split_run/ && rm -rf /home/scratch/{n} \n'
    fout.write(cmd)
fout.close()
```

Note: change cpus to 9, to speed up a little bit.


```bash
srun -N 16 --partition long --qos long-award   -t 240:00:00 --pty bash #allocate 16 nodes
python /home1/xc278/w/GitHub/xiaolongTools/multiThreadSlurm.py -t 9 -n e3c-012,e3c-013,e3c-014,e3c-015,e3c-016,e3c-017,e3c-018,e3c-019,e3c-020,e3c-021,e3c-022,e3c-023,e3c-024,e3c-025,e3c-026,e3c-027 -s 60 -i /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step1/cmds.txt 
```

some scaffolds are too large, too slow to run with single thread. tested, maker can be run with mpi with the code below. 

```python
txt = '''#!/bin/bash

#SBATCH --job-name={n}
#SBATCH --partition=main
#SBATCH --qos main-award 
#SBATCH --time=48:00:00
#SBATCH --error=err.%x.%j.%N
#SBATCH --output=out.%x.%j.%N
#SBATCH --export=ALL
#SBATCH --nodes=1
#SBATCH --ntasks=24
####SBATCH --mem=256GB
#SBATCH --mail-user=xiaolong.cao@rutgers.edu
#SBATCH --mail-type=ALL
#SBATCH --requeue                    # Return job to the queue if preempted
#SBATCH -A jx76-004 #which project acount to charge




source activate /gpfs/gpfs/scratch/xc278/maker2 && cd /home/scratch && mkdir {n} && cd {n} && cp /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step1//maker*.ctl ./ && mpiexec -n 36   maker -genome  /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/scfs/{n}  &&  gff3_merge -d *.maker.output/*master_datastore_index.log &&  mv *.gff /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step1//maker_split_run/ && rm -rf /home/scratch/1 

'''

for i in '1 2 3 4 5 96 97 99 134'.split():
    open(f"{i}.sbatch",'w',newline='\n').write(txt.format(n=i))
    

```

In the future, maybe use mpi maker directly. no need to split the genome, which will be much simpler.

to be compatible with the following step, use python to combine the result

```python
import glob
files = glob.glob('/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step1/maker_split_run/*.gff')
outfile = '/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step1/step1.all.gff'

ls_gff = []
ls_fasta = []
for f in files:
    txt = open(f).read()
    txt = txt.replace('##gff-version 3\n','',1)
    txt_gff, txt_fa = txt.split('##FASTA\n')
    ls_gff.append(txt_gff)
    ls_fasta.append(txt_fa)

fout = open(outfile,'w')
fout.write('##gff-version 3\n')
fout.write(''.join(ls_gff))
fout.write('##FASTA\n')
fout.write(''.join(ls_fasta))
fout.close()
```

### train SNAP

the scaffold name is like 'Scaffold_12;HRSCAF=78', has to change it to like `Scaffold_12_HRSCAF_78`, otherwise the program does not work normally

```python
file = 'step1.all.gff'
txt = open(file).read()
txt = txt.replace(';HRSCAF=','_HRSCAF_')
txt = txt.replace('%3BHRSCAF%3D','_HRSCAF_')
open(file,'w').write(txt)
```

```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step1
source activate /gpfs/gpfs/scratch/xc278/maker2 #activate environment

maker2zff step1.all.gff
fathom genome.ann genome.dna -validate > snap_validate_output.txt
cat snap_validate_output.txt | grep "error" > error.log #5 error MODEL24893 MODEL2504 MODEL2505 MODEL918 MODEL1855
grep -vwE "(MODEL24893|MODEL2504|MODEL2505|MODEL918|MODEL1855)" genome.ann > genome.ann2
cat genome.ann2 |grep "MODEL24893"
cat genome.ann |grep "MODEL24893"
fathom genome.ann2 genome.dna -validate
fathom genome.ann2 genome.dna -categorize 1000
fathom uni.ann uni.dna -export 1000 -plus 
forge export.ann export.dna
hmm-assembler.pl my_genome . > my_genome.hmm
```

### step6 train augustus
```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step1
perl /gpfs/gpfs/scratch/xc278/maker2/bin/zff2augustus_gbk.pl > augustus.gbk
randomSplit.pl augustus.gbk 100
new_species.pl --species=Rdo20190628
etraining --species=Rdo20190628 augustus.gbk.train
augustus --species=Rdo20190628 augustus.gbk.test | tee first_training.out
optimize_augustus.pl --species=Rdo20190628 augustus.gbk.train --cpus=36 --kfold=36

```

### train GeneMark
```
cd /lab02/Data_Raw/Xiaolong/temp
export PERL5LIB=/home/xcao/p/anaconda3/lib/site_perl/5.26.2/
perl /home/xcao/p/gm_et_linux_64/gmes_petap/gmes_petap.pl --cores 48 --ES --sequence Rdo_Scaffolds.fa
```

### step9 maker final run

```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step9/
cp /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step1/maker*.ctl ./
#cp /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/Genome/Rdo_Scaffolds.fa ./
#sed -i -e 's/=/_/g' -e 's/;/_/g' Rdo_Scaffolds.fa #change scaffold name, change '=' and ';' to '_'
```
change scaffold name, and change bases like "Y" in the sequence to "N"
```python

filein = '/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/Genome/Rdo_Scaffolds.fa'
fileout = '/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step9/Rdo_Scaffolds.fa'

from Bio import SeqIO
import re

fout = open(fileout,'w')
for s in SeqIO.parse(filein,'fasta'):
    slen = len(s.seq)
    seq = re.sub('[^ATCG]','N',str(s.seq))
    sid = re.sub('=|;','_', s.id)
    fout.write('>'+sid+'\n'+seq+'\n')
fout.close()
```



change the `maker_opts.ctl` to 
```
#-----Genome (these are always required)
genome=/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step9/Rdo_Scaffolds.fa #genome sequence (fasta file or fasta embeded in GFF3 file)
organism_type=eukaryotic #eukaryotic or prokaryotic. Default is eukaryotic

#-----Re-annotation Using MAKER Derived GFF3
maker_gff=/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step1/step1.all.gff #MAKER derived GFF3 file
est_pass=1 #use ESTs in maker_gff: 1 = yes, 0 = no
altest_pass=1 #use alternate organism ESTs in maker_gff: 1 = yes, 0 = no
protein_pass=1 #use protein alignments in maker_gff: 1 = yes, 0 = no
rm_pass=1 #use repeats in maker_gff: 1 = yes, 0 = no
model_pass=0 #use gene models in maker_gff: 1 = yes, 0 = no
pred_pass=0 #use ab-initio predictions in maker_gff: 1 = yes, 0 = no
other_pass=0 #passthrough anyything else in maker_gff: 1 = yes, 0 = no

#-----EST Evidence (for best results provide a file for at least one)
est= #set of ESTs or assembled mRNA-seq in fasta format
altest= #EST/cDNA sequence file in fasta format from an alternate organism
est_gff= #aligned ESTs or mRNA-seq from an external GFF3 file
altest_gff= #aligned ESTs from a closly relate species in GFF3 format

#-----Protein Homology Evidence (for best results provide a file for at least one)
protein=  #protein sequence file in fasta format (i.e. from mutiple oransisms)
protein_gff=  #aligned protein homology evidence from an external GFF3 file

#-----Repeat Masking (leave values blank to skip repeat masking)
model_org=all #select a model organism for RepBase masking in RepeatMasker
rmlib= #provide an organism specific repeat library in fasta format for RepeatMasker
repeat_protein= #provide a fasta file of transposable element proteins for RepeatRunner
rm_gff= #pre-identified repeat elements from an external GFF3 file
prok_rm=0 #forces MAKER to repeatmask prokaryotes (no reason to change this), 1 = yes, 0 = no
softmask=1 #use soft-masking rather than hard-masking in BLAST (i.e. seg and dust filtering)

#-----Gene Prediction
snaphmm=/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step1/my_genome.hmm #SNAP HMM file
gmhmm=/gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step8/gmhmm.mod #GeneMark HMM file
augustus_species=Rdo20190628 #Augustus gene prediction species model
fgenesh_par_file= #FGENESH parameter file
pred_gff= #ab-initio predictions from an external GFF3 file
model_gff= #annotated gene models from an external GFF3 file (annotation pass-through)
est2genome=1 #infer gene predictions directly from ESTs, 1 = yes, 0 = no
protein2genome=1 #infer predictions from protein homology, 1 = yes, 0 = no
trna=0 #find tRNAs with tRNAscan, 1 = yes, 0 = no
snoscan_rrna= #rRNA file to have Snoscan find snoRNAs
unmask=0 #also run ab-initio prediction programs on unmasked sequence, 1 = yes, 0 = no

#-----Other Annotation Feature Types (features MAKER doesn't recognize)
other_gff= #extra features to pass-through to final MAKER generated GFF3 file

#-----External Application Behavior Options
alt_peptide=C #amino acid used to replace non-standard amino acids in BLAST databases
cpus=1 #max number of cpus to use in BLAST and RepeatMasker (not for MPI, leave 1 when using MPI)

#-----MAKER Behavior Options
max_dna_len=100000 #length for dividing up contigs into chunks (increases/decreases memory usage)
min_contig=1 #skip genome contigs below this length (under 10kb are often useless)

pred_flank=200 #flank for extending evidence clusters sent to gene predictors
pred_stats=0 #report AED and QI statistics for all predictions as well as models
AED_threshold=1 #Maximum Annotation Edit Distance allowed (bound by 0 and 1)
min_protein=0 #require at least this many amino acids in predicted proteins
alt_splice=0 #Take extra steps to try and find alternative splicing, 1 = yes, 0 = no
always_complete=0 #extra steps to force start and stop codons, 1 = yes, 0 = no
map_forward=0 #map names and attributes forward from old GFF3 genes, 1 = yes, 0 = no
keep_preds=0 #Concordance threshold to add unsupported gene prediction (bound by 0 and 1)

split_hit=10000 #length for the splitting of hits (expected max intron size for evidence alignments)
single_exon=1 #consider single exon EST evidence when generating annotations, 1 = yes, 0 = no
single_length=600 #min length required for single exon ESTs if 'single_exon is enabled'
correct_est_fusion=0 #limits use of ESTs in annotation to avoid fusion genes

tries=2 #number of times to try a contig if there is a failure for some reason
clean_try=0 #remove all data from previous run before retrying, 1 = yes, 0 = no
clean_up=0 #removes theVoid directory with individual analysis files, 1 = yes, 0 = no
TMP= #specify a directory other than the system default temporary directory for temporary files

```

run maker

```bash
mpiexec -n 36   maker
```


### clean files

```
mv /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step9/Rdo_Scaffolds.* /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/20190622Rhyzopertha_dominica/
cp /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step1/my_genome.hmm /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/20190622Rhyzopertha_dominica/Rdo.snap.hmm
cp /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/step8/gmhmm.mod /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/20190622Rhyzopertha_dominica/Rdo.genemark.hmm
cp -r /gpfs/gpfs/scratch/xc278/maker2/config/species/Rdo20190628/ /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/20190622Rhyzopertha_dominica/
pigz /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/20190622Rhyzopertha_dominica/Rdo_Scaffolds.all.gff

rm -rf /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/RNA/
rm -rf /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/maker/
```

## BUSCO estimate quality

### BUSCO for Genome

```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/BUSCO/
source activate /gpfs/gpfs/scratch/xc278/maker2/
python /home1/xc278/p/BUSCO/busco-master/scripts/run_BUSCO.py -c 36 -m genome -l /home1/xc278/p/BUSCO/insecta/insecta_odb9/ -i /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/Genome/Rdo_Scaffolds.fa -o 20190625RdoGenome

```

result
```
C:99.4%[S:98.8%,D:0.6%],F:0.2%,M:0.4%,n:1658
```

### BUSCO for augustus protein

```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/BUSCO/
source activate /gpfs/gpfs/scratch/xc278/maker2/
python /home1/xc278/p/BUSCO/busco-master/scripts/run_BUSCO.py -c 36 -m prot -l /home1/xc278/p/BUSCO/insecta/insecta_odb9/ -i /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/Genome/blast2go_fasta_pred_prot.fasta -o 20190625RdoAugustusProt

```

result
```
C:71.9%[S:70.9%,D:1.0%],F:7.8%,M:20.3%,n:1658
```

### BUSCO for StringTie, Trinity and TransAbyss

```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/BUSCO/
source activate /gpfs/gpfs/scratch/xc278/maker2/
python /home1/xc278/p/BUSCO/busco-master/scripts/run_BUSCO.py -c 36 -m tran -l /home1/xc278/p/BUSCO/insecta/insecta_odb9/ -i /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/StringTie/20190625RdoStringTie.fa -o 20190625Rdo.StringTie
python /home1/xc278/p/BUSCO/busco-master/scripts/run_BUSCO.py -c 36 -m tran -l /home1/xc278/p/BUSCO/insecta/insecta_odb9/ -i /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/denovo/20190624Trinity.fasta -o 20190625Rdo.Trinity
python /home1/xc278/p/BUSCO/busco-master/scripts/run_BUSCO.py -c 36 -m tran -l /home1/xc278/p/BUSCO/insecta/insecta_odb9/ -i /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/denovo/20190625transabyss.fasta -o 20190625Rdo.TransAbyss


```

result
```
StringTie: C:98.3%[S:57.0%,D:41.3%],F:0.8%,M:0.9%,n:1658
Trinity: C:94.6%[S:58.3%,D:36.3%],F:4.6%,M:0.8%,n:1658
TransAbyss: C:96.0%[S:94.9%,D:1.1%],F:3.7%,M:0.3%,n:1658

```

### BUSCO for maker protein sets

maker output 5 protein files. 
```
Rdo_Scaffolds.all.maker.augustus_masked.proteins.fasta
Rdo_Scaffolds.all.maker.genemark.proteins.fasta
Rdo_Scaffolds.all.maker.non_overlapping_ab_initio.proteins.fasta
Rdo_Scaffolds.all.maker.proteins.fasta
Rdo_Scaffolds.all.maker.snap_masked.proteins.fasta
```

```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/BUSCO/
source activate /gpfs/gpfs/scratch/xc278/maker2/
python /home1/xc278/p/BUSCO/busco-master/scripts/run_BUSCO.py -c 36 -m prot -l /home1/xc278/p/BUSCO/insecta/insecta_odb9/ -i /gpfs/gpfs/staging/jx76-003/xc/20190622Rhyzopertha_dominica/20190622Rhyzopertha_dominica/Rdo_Scaffolds.all.maker.proteins.fasta -o 20190629RdoMakerProt

```

result
```
Rdo_Scaffolds.all.maker.proteins.fasta    C:92.1%[S:91.1%,D:1.0%],F:2.3%,M:5.6%,n:1658
Rdo_Scaffolds.all.maker.augustus_masked.proteins.fasta        C:94.4%[S:93.3%,D:1.1%],F:3.6%,M:2.0%,n:1658
Rdo_Scaffolds.all.maker.genemark.proteins.fasta        C:86.9%[S:85.8%,D:1.1%],F:7.8%,M:5.3%,n:1658
Rdo_Scaffolds.all.maker.non_overlapping_ab_initio.proteins.fasta        C:2.8%[S:2.7%,D:0.1%],F:2.1%,M:95.1%,n:1658
Rdo_Scaffolds.all.maker.snap_masked.proteins.fasta        C:72.2%[S:71.2%,D:1.0%],F:10.6%,M:17.2%,n:1658
```

## MCuNovo

### translate with Transdecoder

```bash
cd /mnt/d/species/genomes/20190622Rhyzopertha_dominica/StringTie/
~/p/transdecoder/TransDecoder-TransDecoder-v5.5.0/TransDecoder.LongOrfs -t /mnt/d/species/genomes/20190622Rhyzopertha_dominica/StringTie/20190625RdoStringTie.fa -m 60 -O /mnt/d/species/genomes/20190622Rhyzopertha_dominica/StringTie/transdecoder
~/p/transdecoder/TransDecoder-TransDecoder-v5.5.0/TransDecoder.Predict -t /mnt/d/species/genomes/20190622Rhyzopertha_dominica/StringTie/20190625RdoStringTie.fa  -O /mnt/d/species/genomes/20190622Rhyzopertha_dominica/StringTie/transdecoder

cd /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/
~/p/transdecoder/TransDecoder-TransDecoder-v5.5.0/TransDecoder.LongOrfs -t /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/20190624Trinity.fasta -m 60 -O /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/20190624Trinity.transdecoder
~/p/transdecoder/TransDecoder-TransDecoder-v5.5.0/TransDecoder.Predict -t /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/20190624Trinity.fasta  -O /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/20190624Trinity.transdecoder

cd /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/
~/p/transdecoder/TransDecoder-TransDecoder-v5.5.0/TransDecoder.LongOrfs -t /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/20190625transabyss.fasta -m 60 -O /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/20190625transabyss.transdecoder
~/p/transdecoder/TransDecoder-TransDecoder-v5.5.0/TransDecoder.Predict -t /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/20190625transabyss.fasta  -O /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/20190625transabyss.transdecoder

```

### Remove duplicated protein sequences

```bash
cd /mnt/d/species/genomes/20190622Rhyzopertha_dominica/StringTie/
python /mnt/c/Users/ATPs/Documents/GitHub/MCuNovo/tools/fastaFileRemoveDup.py 20190625RdoStringTie.fa.transdecoder.pep

cd /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/
python /mnt/c/Users/ATPs/Documents/GitHub/MCuNovo/tools/fastaFileRemoveDup.py 20190624Trinity.fasta.transdecoder.pep

cd /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/
python /mnt/c/Users/ATPs/Documents/GitHub/MCuNovo/tools/fastaFileRemoveDup.py 20190625transabyss.fasta.transdecoder.pep
```


### blast against uniprot

Note: it seems blastp-fast is not so sensitive. use blastp if possible.
```bash
#download fasta file of arthropoda fasta from uniprot
cd /mnt/d/species/uniprot/arthropoda
wget ftp://ftp.ebi.ac.uk/pub/databases/fastafiles/uniprot/uniprotkb_arthropoda.gz
gzip -d uniprotkb_arthropoda.gz

#remove sequences that are labeled as fragment and with X
python /mnt/c/Users/ATPs/Documents/GitHub/MCuNovo/tools/fastaFileClean.py -i uniprotkb_arthropoda

#build database
~/p/blast/ncbi-blast-2.9.0+/bin/segmasker -in uniprotkb_arthropodaclean -infmt fasta -parse_seqids -outfmt maskinfo_asn1_bin -out uniprotkb_arthropodaclean.asnb
~/p/blast/ncbi-blast-2.9.0+/bin/makeblastdb -in uniprotkb_arthropodaclean -input_type fasta -dbtype prot -parse_seqids -mask_data uniprotkb_arthropodaclean.asnb -out uniprotkb_arthropodacleanDB -title 'uniprotkb_arthropodaclean20190714'

#run blast
~/p/blast/ncbi-blast-2.9.0+/bin/blastp -db /mnt/d/species/uniprot/arthropoda/uniprotkb_arthropodacleanDB -query /mnt/d/species/genomes/20190622Rhyzopertha_dominica/StringTie/20190625RdoStringTie.fa.transdecoder.pepUN -out /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190714MCuNovo/StringTie2U -outfmt 6 -num_threads 6 -max_target_seqs 20 -task blastp-fast  

~/p/blast/ncbi-blast-2.9.0+/bin/blastp -db /mnt/d/species/uniprot/arthropoda/uniprotkb_arthropodacleanDB -query /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/20190624Trinity.fasta.transdecoder.pepUN -out /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190714MCuNovo/Trinity2U -outfmt 6 -num_threads 6 -max_target_seqs 20  -task blastp-fast  

~/p/blast/ncbi-blast-2.9.0+/bin/blastp -db /mnt/d/species/uniprot/arthropoda/uniprotkb_arthropodacleanDB -query /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/20190625transabyss.fasta.transdecoder.pepUN -out /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190714MCuNovo/TransAbyss2U -outfmt 6 -num_threads 6 -max_target_seqs 20 -task blastp-fast  

~/p/blast/ncbi-blast-2.9.0+/bin/blastp -db /mnt/d/species/uniprot/arthropoda/uniprotkb_arthropodacleanDB -query /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190622Rhyzopertha_dominica/Rdo_Scaffolds.all.maker.proteins.fasta -out /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190714MCuNovo/Maker2U -outfmt 6 -num_threads 6 -max_target_seqs 20 -task blastp-fast  

```

### parse blast6 to calculate match length

```bash
cd /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190714MCuNovo
python /mnt/c/Users/ATPs/Documents/GitHub/MCuNovo/tools/parseBlast6.py -i /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190714MCuNovo/StringTie2U -o /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190714MCuNovo/StringTie2U.match -m 30 -q /mnt/d/species/genomes/20190622Rhyzopertha_dominica/StringTie/20190625RdoStringTie.fa.transdecoder.pepUN -s /mnt/d/species/uniprot/arthropoda/uniprotkb_arthropodaclean

python /mnt/c/Users/ATPs/Documents/GitHub/MCuNovo/tools/parseBlast6.py -i /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190714MCuNovo/Trinity2U -o /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190714MCuNovo/Trinity2U.match -m 30 -q /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/20190624Trinity.fasta.transdecoder.pepUN -s /mnt/d/species/uniprot/arthropoda/uniprotkb_arthropodaclean

python /mnt/c/Users/ATPs/Documents/GitHub/MCuNovo/tools/parseBlast6.py -i /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190714MCuNovo/TransAbyss2U -o /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190714MCuNovo/TransAbyss2U.match -m 30 -q /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/20190625transabyss.fasta.transdecoder.pepUN -s /mnt/d/species/uniprot/arthropoda/uniprotkb_arthropodaclean

python /mnt/c/Users/ATPs/Documents/GitHub/MCuNovo/tools/parseBlast6.py -i /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190714MCuNovo/Maker2U -o /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190714MCuNovo/Maker2U.match -m 30 -q /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190622Rhyzopertha_dominica/Rdo_Scaffolds.all.maker.proteins.fasta -s /mnt/d/species/uniprot/arthropoda/uniprotkb_arthropodaclean
```

### compare between four set of proteins
```bash
cd /mnt/d/species/genomes/20190622Rhyzopertha_dominica

python /mnt/c/Users/ATPs/Documents/GitHub/MCuNovo/tools/SeqCompare.py -q ./20190622Rhyzopertha_dominica/Rdo_Scaffolds.all.maker.proteins.fasta -s ./StringTie/20190625RdoStringTie.fa.transdecoder.pepUN -o ./20190714MCuNovo/maker2StringTie -p ~/p/muscle/muscle

python /mnt/c/Users/ATPs/Documents/GitHub/MCuNovo/tools/SeqCompare.py -q ./20190622Rhyzopertha_dominica/Rdo_Scaffolds.all.maker.proteins.fasta -s ./denovo/20190624Trinity.fasta.transdecoder.pepUN -o ./20190714MCuNovo/maker2Trinity -p ~/p/muscle/muscle

python /mnt/c/Users/ATPs/Documents/GitHub/MCuNovo/tools/SeqCompare.py -q ./20190622Rhyzopertha_dominica/Rdo_Scaffolds.all.maker.proteins.fasta -s ./denovo/20190625transabyss.fasta.transdecoder.pepUN -o ./20190714MCuNovo/maker2TransAbyss -p ~/p/muscle/muscle

python /mnt/c/Users/ATPs/Documents/GitHub/MCuNovo/tools/SeqCompare.py -q ./StringTie/20190625RdoStringTie.fa.transdecoder.pepUN -s ./denovo/20190624Trinity.fasta.transdecoder.pepUN -o ./20190714MCuNovo/StringTie2Trinity -p ~/p/muscle/muscle

python /mnt/c/Users/ATPs/Documents/GitHub/MCuNovo/tools/SeqCompare.py -q ./StringTie/20190625RdoStringTie.fa.transdecoder.pepUN -s ./denovo/20190625transabyss.fasta.transdecoder.pepUN -o ./20190714MCuNovo/StringTie2TransAbyss -p ~/p/muscle/muscle


```

### use blast to compare between four set of proteins

use blast for reference in case there are some wired things. Not used for MCuNovo. To speed up, run on HPC
```python
import os

os.chdir('/cache/home/xc278/w/GitHub/temp/')


files = '''/cache/home/xc278/w/GitHub/temp/20190624Trinity.fasta.transdecoder.pepUN
/cache/home/xc278/w/GitHub/temp/20190625RdoStringTie.fa.transdecoder.pepUN
/cache/home/xc278/w/GitHub/temp/20190625transabyss.fasta.transdecoder.pepUN
/cache/home/xc278/w/GitHub/temp/Rdo_Scaffolds.all.maker.proteins.fasta
'''.split()

keys = 'Trinity StringTie TransAbyss maker'.split()
dc_files = dict(zip(keys, files))

makeblastdb = '/cache/sw/packages/blast/2.6.0/ncbi-blast-2.6.0+/bin/makeblastdb'
blastp = '/cache/sw/packages/blast/2.6.0/ncbi-blast-2.6.0+/bin/blastp'

cmd_makedb = '''{makeblastdb} -in {protein_file} -input_type fasta -dbtype prot -parse_seqids -out {key} -title '{key}' &'''

for k, protein_file in dc_files.items():
    os.system(cmd_makedb.format(key=k, protein_file=protein_file, makeblastdb=makeblastdb))

cmd_blast = '''cd /cache/home/xc278/w/GitHub/temp/ && {blastp} -db {subject} -query {protein_file} -out {query}2{subject} -outfmt 6 -num_threads 24 -task blastp-fast -max_target_seqs 20 &'''

qs_pairs = []
qs_pairs.append(['maker', 'StringTie'])
qs_pairs.append(['maker', 'Trinity'])
qs_pairs.append(['maker', 'TransAbyss'])
qs_pairs.append(['StringTie', 'Trinity'])
qs_pairs.append(['StringTie','TransAbyss'])
for query, subject in qs_pairs:
    print(cmd_blast.format(blastp=blastp, subject=subject, query=query, protein_file=dc_files[query]))
```

### generate a table, with protein_id, transcript_id and gene_id

```bash
python /mnt/c/Users/ATPs/Documents/GitHub/MCuNovo/tools/Protein2Transcript2Gene.py -c /mnt/d/species/genomes/20190622Rhyzopertha_dominica//StringTie/20190625RdoStringTie.fa /mnt/d/species/genomes/20190622Rhyzopertha_dominica/StringTie/20190625RdoStringTie.fa.transdecoder.pepUN -o /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190714MCuNovo/protein2transcript2gene.txt -d /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/20190624Trinity.fasta.transdecoder.pepUN /mnt/d/species/genomes/20190622Rhyzopertha_dominica/denovo/20190625transabyss.fasta.transdecoder.pepUN -m /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190622Rhyzopertha_dominica/Rdo_Scaffolds.all.maker.proteins.fasta
```

### generate a table of proteins with 'X'
```bash
python  /mnt/c/Users/ATPs/Documents/GitHub/MCuNovo/tools/proteinXFinder.py -i /mnt/d/species/genomes/20190622Rhyzopertha_dominica/StringTie/20190625RdoStringTie.fa.transdecoder.pepUN /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190622Rhyzopertha_dominica/Rdo_Scaffolds.all.maker.proteins.fasta -o /mnt/d/species/genomes/20190622Rhyzopertha_dominica/20190714MCuNovo/proteinXcount.txt
```

### run selector.
check the C:\Users\ATPs\Documents\GitHub\MCuNovo\tools\SelectorBasedOnMatch.py file

### run BUSCO

```
cd /projectsp/f_jx76_1/xiaolong/temp
python /cache/home/xc278/p/busco/scripts/run_BUSCO.py -c 24 -m prot -o 20190902  -l /cache/home/xc278/p/busco/insecta_odb9/ -i MCD.MCD.protein.txt
```

output

```
INFO    C:99.8%[S:90.0%,D:9.8%],F:0.1%,M:0.1%,n:1658
INFO    1655 Complete BUSCOs (C)
INFO    1492 Complete and single-copy BUSCOs (S)
INFO    163 Complete and duplicated BUSCOs (D)
INFO    2 Fragmented BUSCOs (F)
INFO    1 Missing BUSCOs (M)

```